In [179]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import sys

# read single file
df = pd.read_csv("new_sample_7_26/files_new/20190617232649_allsites.csv", sep="\t")
# select variable
df = df[["染色体位置","HGVS","REVEL/M-CAP",
                   "SIFT score","Polyphen2 score","表型相关度",
                   "Clinvar","HGMD","ExonicFunc_refGene","系统结论","FinalResult","user_confirm"]]


In [180]:
# drop mistake lines in appending original files 
df = df.loc[df["HGMD"] != "HGMD"]
# combine 染色体位置 and HGVS as a new variable
df["loc_HGVS"] = df["染色体位置"].map(str) + df["HGVS"]
df = df.loc[df["loc_HGVS"].notna()]
# check duplicates
dup = df["loc_HGVS"].value_counts()
# delete duplicates by 染色体位置+HGVS
# note: len = 820159 after this step 
df_filtered = df.drop_duplicates("loc_HGVS")
# save duplicates as a col in df
df_dup = pd.DataFrame(dup)
# index = loc_HGVS, added counts in df
df_filtered = df_filtered.set_index("loc_HGVS")
df_filtered["count"] = df_dup

# split REVEL/M-CAP, S/p/M, drop original
df_filtered[['REVEL','M-CAP']]= df_filtered["REVEL/M-CAP"].str.split('/',expand = True)
df_filtered = df_filtered.drop(['REVEL/M-CAP'], 1)
# remove non-meaningful values "-" and "-1.0"
# note: "-" and "-1.0" means NaN in upstream analysis
df_filtered = df_filtered.replace('-', np.nan)
df_filtered = df_filtered.replace('-1.0', np.nan)

# filter out meaningless data
# note: len = 363511 after this step
df_filtered = df_filtered.loc[(df_filtered["SIFT score"].notna()) |
                            (df_filtered["Polyphen2 score"].notna()) |
                            (df_filtered["表型相关度"].notna()) |
                            (df_filtered["Clinvar"].notna()) |
                            (df_filtered["HGMD"].notna()) |
                            (df_filtered["REVEL"].notna()) |
                            (df_filtered["M-CAP"].notna())]

In [181]:
# Clinvar quanlification
Clinvar_result = [None]* len(df_filtered)
count = 0
for i in df_filtered["Clinvar"]:
    if "likely" in str(i) and "benign" in str(i) and "/" in str(i) :
        Clinvar_result[count]= "C_benign/likely_benign"
    elif "likely" in str(i) and "benign" in str(i):
        Clinvar_result[count]= "C_likely_benign"
    elif "benign" in str(i):
        Clinvar_result[count]= "C_benign"
    elif "uncertain" in str(i) and "significance" in str(i):
        Clinvar_result[count]= "C_uncertain_significance"
    elif "conflicting" in str(i) and "interpretations" in str(i):
        Clinvar_result[count]= "C_conflicting_interpretations"
    elif "not" in str(i) and "provided" in str(i):
        Clinvar_result[count]= "C_not_provided"
    elif "likely" in str(i) and "pathogenic" in str(i) and "/" in str(i):
        Clinvar_result[count]= "C_pathogenic/likely_pathogenic"
    elif "likely" in str(i) and "pathogenic" in str(i):
        Clinvar_result[count]= "C_likely_pathogenic"
    elif "pathogenic" in str(i):
        Clinvar_result[count]= "C_pathogenic"
    elif "association" in str(i):
        Clinvar_result[count]= "C_association"
    else:
        Clinvar_result[count]= "C_NaN"
    count += 1

df_filtered["Clinvar"] = Clinvar_result

# 系统结论量化
result_result = [None]* len(df_filtered)
count = 0
for i in df_filtered["系统结论"]:
    if str(i) == "Benign":
        result_result[count]= 0
    elif str(i) == "Likely benign" :
        result_result[count]= 1
    elif str(i) == "Uncertain significance" :
        result_result[count]= 2
    elif str(i) == "Likely pathogenic" :
        result_result[count]= 3
    elif str(i) == "Pathogenic" :
        result_result[count]= 4
    count += 1



In [182]:
# transfer categorical to dummy variables
df_Clinvar_dummy = pd.get_dummies(df_filtered["Clinvar"])
df_filtered["ExonicFunc_refGene"] = df_filtered["ExonicFunc_refGene"].fillna("NaN_ExonicFunc")
df_ExonicFunc_dummy = pd.get_dummies(df_filtered["ExonicFunc_refGene"])
df_filtered["HGMD"] = df_filtered["HGMD"].fillna("NaN_HGMD")
df_HGMD_dummy = pd.get_dummies(df_filtered["HGMD"])


# new df to save training data
df_model = pd.DataFrame()
# location information
df_model["loc"] = df_filtered["染色体位置"]
df_model["HGVS"] = df_filtered["HGVS"]
# outcome
df_model["result"] = result_result
df_model["user_confirm"] = pd.to_numeric(df_filtered["user_confirm"])
# annotations
df_model["SIFT"] = pd.to_numeric(df_filtered["SIFT score"])
df_model["SIFT"] = df_model["SIFT"].fillna(df_model["SIFT"].mean())
df_model["polyphen2"] = pd.to_numeric(df_filtered["Polyphen2 score"])
df_model["polyphen2"] = df_model["polyphen2"].fillna(df_model["polyphen2"].mean())
df_model["REVEL"] = pd.to_numeric(df_filtered["REVEL"])
df_model["REVEL"] = df_model["REVEL"].fillna(df_model["REVEL"].mean())
df_model["M-CAP"] = pd.to_numeric(df_filtered["M-CAP"])
df_model["M-CAP"] = df_model["M-CAP"].fillna(df_model["M-CAP"].mean())
df_model["count"] = pd.to_numeric(df_filtered["count"])
df_model["pheno_score"] = pd.to_numeric(df_filtered["表型相关度"])
df_model["pheno_score"] = df_model["pheno_score"].fillna(df_model["pheno_score"].mean())
df_model["pheno_score"] = df_model["pheno_score"].fillna(0)

# join dummy df to df_model
df_model = df_model.join(df_Clinvar_dummy, how='outer')
df_model = df_model.join(df_ExonicFunc_dummy, how='outer')
df_model = df_model.join(df_HGMD_dummy, how='outer')

In [183]:
for i in ["SIFT","polyphen2","REVEL","M-CAP","count","pheno_score",
                             "C_NaN","C_association","C_benign","C_benign/likely_benign","C_conflicting_interpretations",
                            "C_likely_benign","C_likely_pathogenic","C_not_provided","C_pathogenic",
                             "C_pathogenic/likely_pathogenic", "C_uncertain_significance", "NaN_ExonicFunc",
                            "frameshift deletion", "frameshift insertion", "frameshift substitution",
                            "nonframeshift deletion", "nonframeshift insertion", "nonframeshift substitution",
                            "nonsynonymous SNV", "stopgain","stoploss","synonymous SNV", "unknown","DFP","DM","DM?","DP",
                            "FP","NaN_HGMD","R"]:
    if i not in df_model.columns:
        df_model[i] = 0
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4333 entries, chr1:161483723NM_001136219:exon6:c.780+1G>A to chrX:70390928NM_001166660:c.*981delT
Data columns (total 40 columns):
loc                               4333 non-null object
HGVS                              4333 non-null object
result                            4333 non-null int64
user_confirm                      4333 non-null int64
SIFT                              4333 non-null float64
polyphen2                         4333 non-null float64
REVEL                             4333 non-null float64
M-CAP                             4333 non-null float64
count                             4333 non-null int64
pheno_score                       4333 non-null float64
C_NaN                             4333 non-null uint8
C_association                     4333 non-null uint8
C_benign                          4333 non-null uint8
C_benign/likely_benign            4333 non-null uint8
C_conflicting_interpretations     4333 non-null uint8
C_

In [184]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn import metrics
from sklearn.externals import joblib

df_model_mean_l_b_p = df_model.loc[df_model["result"]>=0]
df_model_mean_l_b_p["p_or_not"] = np.nan
df_model_mean_l_b_p.loc[df_model_mean_l_b_p.result < 3, 'p_or_not'] = "0"
df_model_mean_l_b_p.loc[df_model_mean_l_b_p.result >= 3, 'p_or_not'] = "1"
df_model_mean_l_b_p.loc[df_model_mean_l_b_p.user_confirm < 1, 'c_or_not'] = "0"
df_model_mean_l_b_p.loc[df_model_mean_l_b_p.user_confirm >= 1, 'c_or_not'] = "1"

X = df_model_mean_l_b_p[["SIFT","polyphen2","REVEL","M-CAP","count","pheno_score",
                             "C_NaN","C_association","C_benign","C_benign/likely_benign","C_conflicting_interpretations",
                            "C_likely_benign","C_likely_pathogenic","C_not_provided","C_pathogenic",
                             "C_pathogenic/likely_pathogenic", "C_uncertain_significance", "NaN_ExonicFunc",
                            "frameshift deletion", "frameshift insertion", "frameshift substitution",
                            "nonframeshift deletion", "nonframeshift insertion", "nonframeshift substitution",
                            "nonsynonymous SNV", "stopgain","stoploss","synonymous SNV", "unknown","DFP","DM","DM?","DP",
                            "FP","NaN_HGMD","R"]]
y_test = df_model_mean_l_b_p["p_or_not"]

# load scalar from disk
loaded_scalar = joblib.load("p_or_not_lr_scalar.sav")
X_std = loaded_scalar.transform(X)


# load model from disk
loaded_model = joblib.load("p_or_not_lr_model.sav")


In [185]:
predictions = loaded_model.predict(X_std)
df_mean_lr_pred = pd.DataFrame(y_test)
df_mean_lr_pred["p_pred"] = predictions
print(len(df_mean_lr_pred.loc[df_mean_lr_pred.iloc[:,1] == "1"])/len(df_mean_lr_pred.loc[df_mean_lr_pred.iloc[:,0] == "1"]))
print("sensitivity of user marked or not ")

20.105263157894736
sensitivity of user marked or not 


In [186]:
df_mean_lr_pred["p_or_not"].value_counts()

0    4314
1      19
Name: p_or_not, dtype: int64

In [187]:
df_mean_lr_pred["p_pred"].value_counts()

KeyError: 'pred'

In [188]:
from sklearn.metrics import confusion_matrix
y_true = df_mean_lr_pred["p_or_not"]
y_pred = df_mean_lr_pred["p_pred"]
p_or_not_tn, p_or_not_fp, p_or_not_fn, p_or_not_tp = confusion_matrix(y_true, y_pred).ravel()
p_or_not_sensitivity = p_or_not_tp / (p_or_not_tp+p_or_not_fn)
p_or_not_specificity = p_or_not_tn / (p_or_not_tn+p_or_not_fp)

df_pred = df_mean_lr_pred

In [204]:
df_mean_lr_pred.loc[df_mean_lr_pred["p_pred"] == "1"]
df_mean_lr_pred.loc[df_mean_lr_pred["p_or_not"] == "1"]

KeyError: 'p_pred'

In [190]:
y_test = df_model_mean_l_b_p["c_or_not"]

# load scalar from disk
loaded_scalar = joblib.load("c_or_not_lr_scalar.sav")
X_std = loaded_scalar.transform(X)


# load model from disk
loaded_model = joblib.load("c_or_not_lr_model.sav")
result = loaded_model.score(X_std, y_test)
print(result)

0.9328409877682898


In [196]:
predictions = loaded_model.predict(X_std)
df_mean_lr_pred = pd.DataFrame(y_test)
df_mean_lr_pred["c_pred"] = predictions
print(len(df_mean_lr_pred.loc[df_mean_lr_pred.iloc[:,1] == "1"])/len(df_mean_lr_pred.loc[df_mean_lr_pred.iloc[:,0] == "1"]))
print("sensitivity of user marked or not ")

98.0
sensitivity of user marked or not 


In [198]:
df_mean_lr_pred.loc[df_mean_lr_pred["c_pred"] == "1"]
df_mean_lr_pred.loc[df_mean_lr_pred["c_or_not"] == "1"]

,c_or_not,c_pred
loc_HGVS,,
chr11:111922043DLAT:NM_001931:exon11:c.1484A>G:p.N495S,1,1
chr16:47630430PHKB:NM_001031835:exon14:c.1330G>A:p.A444T,1,1
chr7:103275986RELN:NM_005045:exon19:c.2351C>T:p.T784M,1,1


In [199]:
df_mean_lr_pred["c_or_not"].value_counts()

0    4330
1       3
Name: c_or_not, dtype: int64

In [200]:
df_mean_lr_pred["c_pred"].value_counts()

0    4039
1     294
Name: c_pred, dtype: int64

In [201]:
from sklearn.metrics import confusion_matrix
y_true = df_mean_lr_pred["c_or_not"]
y_pred = df_mean_lr_pred["c_pred"]
c_or_not_tn, c_or_not_fp, c_or_not_fn, c_or_not_tp = confusion_matrix(y_true, y_pred).ravel()

c_or_not_sensitivity = c_or_not_tp / (c_or_not_tp+c_or_not_fn)
c_or_not_specificity = c_or_not_tn / (c_or_not_tn+c_or_not_fp)

df_pred["c_pred"] = y_pred

In [202]:
df_pred

,p_or_not,p_pred,c_pred
loc_HGVS,,,
chr1:161483723NM_001136219:exon6:c.780+1G>A,0,0,1
chr1:201286861PKP1:NM_000299:exon5:c.1008C>T:p.S336S,0,0,0
chr1:168511325XCL2:NM_003175:exon2:c.82C>G:p.H28D,0,0,0
chr1:227920165JMJD4:NM_023007:exon6:c.1320C>T:p.S440S,0,0,0
chr1:201755636NAV1:NM_020443:exon9:c.2926C>A:p.Q976K,0,0,0
chr1:161487863FCGR2A:NM_001136219:exon7:c.879C>T:p.P293P,0,0,0
chr1:146398387NBPF12:NM_001278141:exon9:c.373G>C:p.E125Q,0,0,0
chr1:168511321XCL2:NM_003175:exon2:c.86G>A:p.R29K,0,0,0
chr1:148004584NBPF26:NM_001351372:exon30:c.3975C>T:p.H1325H,0,0,0


In [106]:
fp

3173

In [107]:
fn

1

In [108]:
tp

2

In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11979 entries, 0 to 11978
Data columns (total 13 columns):
染色体位置                 11979 non-null object
HGVS                  11979 non-null object
REVEL/M-CAP           11979 non-null object
SIFT score            3098 non-null float64
Polyphen2 score       3098 non-null float64
表型相关度                 0 non-null float64
Clinvar               1074 non-null object
HGMD                  295 non-null object
ExonicFunc_refGene    5693 non-null object
系统结论                  11979 non-null object
FinalResult           11979 non-null object
user_confirm          11979 non-null int64
loc_HGVS              11979 non-null object
dtypes: float64(3), int64(1), object(9)
memory usage: 1.3+ MB
